### Setup and Imports

In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path.cwd().parent))
import config
from src import data_ingestion
from src.symbols import Symbols
from src.portfolio1 import Portfolio
from src.benchmark import Benchmark

import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### Load Transaction Data

In [2]:
# Create a list of all your transaction log files
log_files = [
    config.TRANS_LOG_DIR / "us_mkt_transactions.csv",
    config.TRANS_LOG_DIR / "exus_mkt_transactions.csv",
]

# Load and merge all found logs into a single DataFrame
master_log = data_ingestion.create_master_log(log_files).set_index("Date")
display(master_log)

,Type,Symbol,Quantity,Price,Amount,Trading Cost,Currency,Description,Exchange,Source
Date,,,,,,,,,,
2023-02-16,Net Deposit,NaN,NaN,NaN,271.57,NaN,USD,ID: 37fdafdc-d707-42fa-ba74-98b30cf9ab2a - DT2...,US Market,Sarwa Trade
2023-02-17,buy,VOO,0.218967,372.07,-81.47,NaN,USD,Trade Entry,NYSEArca,Sarwa Trade
2023-02-17,buy,TSLA,0.202666,201.02,-40.74,NaN,USD,Trade Entry,NasdaqGS,Sarwa Trade
2023-02-17,buy,PERI,0.407270,33.34,-13.58,NaN,USD,Trade Entry,NasdaqGS,Sarwa Trade
2023-03-29,Net Dividend,VOO,NaN,NaN,0.24,NaN,USD,"Cash DIV @ 1.4874, Pos QTY: 0.218966527, Rec D...",NYSEArca,Sarwa Trade
...,...,...,...,...,...,...,...,...,...,...
2025-07-31,Net Dividend,GOF,NaN,NaN,0.39,NaN,USD,"Cash DIV @ 0.1821, Pos QTY: 3, Rec Date: 2025-...",NYSE,Sarwa Trade
2025-07-31,Net Dividend,BRW,NaN,NaN,0.28,NaN,USD,"Cash DIV @ 0.085, Pos QTY: 4.707717967, Rec Da...",NYSE,Sarwa Trade
2025-08-04,buy,VOO,0.426862,575.08,-245.48,1.0,USD,Purchase of VOO shares on 2025-08-04,NYSEArca,Sarwa Trade - Interim


### Initialize and Assess Symbols

In [3]:
# 1. Initialize the Symbol manager
symbol_manager = Symbols(master_log)

# 2. Assess all symbols (checks cache and yfinance, creates templates for missing symbols)
symbol_manager.assess()

# 3. Display the results for user verification
print("✅ The following symbols were automatically identified:")
found_df = symbol_manager.get_found()
display(found_df)

print("❌ The following symbols were NOT found:")
missing_list = symbol_manager.get_missing()
print(missing_list)

✅ The following symbols were automatically identified:


,Name,Type,Exchange,Currency,Industry,Sector,Country,DataProvider
Symbol,,,,,,,,
PERI,Perion Network Ltd.,equity,NasdaqGS,USD,Internet Content & Information,Communication Services,Israel,yfinance
TSLA,"Tesla, Inc.",equity,NasdaqGS,USD,Auto Manufacturers,Consumer Cyclical,United States,yfinance
VOO,Vanguard S&P 500 ETF,etf,NYSEArca,USD,None,None,None,yfinance
UAE,iShares MSCI UAE ETF,etf,NasdaqGM,USD,None,None,None,yfinance
XLK,The Technology Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,None,yfinance
...,...,...,...,...,...,...,...,...
JPMB,JPMorgan USD Emerging Markets Sovereign Bond ETF,etf,NYSEArca,USD,None,None,None,yfinance
SMOT,VanEck Morningstar SMID Moat ETF,etf,Cboe US,USD,None,None,None,yfinance
XLV,The Health Care Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,None,yfinance


❌ The following symbols were NOT found:
['ALDAR', 'BURJEEL', 'CHADX15', 'IHC', 'EMAAR', 'MULTIPLY']


### User Correction Step

Reviews the output from the cell above and lists any symbols that were incorrectly identified by the data provider (e.g. `yfinance`).

In [4]:
# User reviews the table above and populates this list
incorrectly_identified_symbols = ["AMR", "ASM"]

# Mark these symbols as user-provided, which also updates the metadata template
symbol_manager.mark_as_manual(incorrectly_identified_symbols)

Updating cache for incorrectly identified symbols: ['AMR', 'ASM']
Caches updated successfully.


### Manual Task

1) Open the file: `data/manual-source/metadata.json`.
2) Fill in the `null` values for all the symbols listed.
3) For each of those same symbols, add a price history CSV file to the `data/manual-source/` directory.

### Reload Metadata and Get Final DataFrame

In [5]:
# Reload the user-edited file from disk
symbol_manager.reload_user_metadata()

# Get the final, combined DataFrame of all symbol metadata
symbol_df = symbol_manager.get_unified_df()

print("--- Final Unified Symbol Metadata ---")
display(symbol_df)

--- Final Unified Symbol Metadata ---


,Name,Type,Exchange,Currency,Industry,Sector,Country,DataProvider
Symbol,,,,,,,,
PERI,Perion Network Ltd.,equity,NasdaqGS,USD,Internet Content & Information,Communication Services,Israel,yfinance
TSLA,"Tesla, Inc.",equity,NasdaqGS,USD,Auto Manufacturers,Consumer Cyclical,United States,yfinance
VOO,Vanguard S&P 500 ETF,etf,NYSEArca,USD,None,None,None,yfinance
UAE,iShares MSCI UAE ETF,etf,NasdaqGM,USD,None,None,None,yfinance
XLK,The Technology Select Sector SPDR Fund,etf,NYSEArca,USD,None,None,None,yfinance
...,...,...,...,...,...,...,...,...
IHC,International Holding Company PJSC,equity,ADX,AED,Industrials,Conglomerates,United Arab Emirates,manual
EMAAR,Emaar Properties PJSC,equity,DFM,AED,Real Estate - Development,Real Estate,United Arab Emirates,manual
MULTIPLY,Multiply Group PJSC,equity,ADX,AED,Financials,Asset Management,United Arab Emirates,manual


### Initialize and Run Portfolio Analysis

In [6]:
# Get the project date range
start_date, end_date, date_range, last_market_day = config.project_dates(
    master_log.index
)

# 1. Initialize the Portfolio analysis engine
portfolio = Portfolio(master_log, symbol_manager, date_range, last_market_day)

# 2. Run all calculations (fetching prices, calculating holdings and value)
portfolio.calculate_holdings_and_value()

portfolio_value = portfolio.get_total_value_history()

### Display Portfolio Results

In [7]:
portfolio_value

Date
2023-02-16       0.000000
2023-02-17     137.713026
2023-02-18     137.713026
2023-02-19     137.713026
2023-02-20     137.713026
                 ...     
2025-08-04    3506.821360
2025-08-05    3490.529341
2025-08-06    3511.104812
2025-08-07    3492.055927
2025-08-08    3510.808929
Length: 905, dtype: float64

In [8]:
portfolio.get_current_holdings().round(2).sort_values(
    "Market Value (USD)", ascending=False
)

,Shares,Market Value (USD)
VOO,1.98,1159.14
XLK,1.22,325.45
VOOG,0.72,299.22
MOAT,1.40,134.53
MSFT,0.26,133.56
SPY,0.18,113.21
META,0.14,106.61
CHADX15,106.00,104.48
APO,0.53,75.91
XLF,1.31,67.90


In [9]:
portfolio.get_return_summary().round(2)

,Income,Realized Gains,Unrealized Gains,Total Return
VOO,8.74,38.58,126.27,173.59
XLK,2.65,53.33,70.61,126.59
VOOG,1.24,15.44,56.49,73.17
META,0.30,0.00,64.27,64.57
SPY,1.71,0.00,35.68,37.39
...,...,...,...,...
BURJEEL,0.18,0.00,-6.21,-6.03
TQQQ,0.00,-6.11,0.00,-6.11
EL,0.15,-6.54,0.00,-6.39
ROM,0.00,-8.08,0.00,-8.08


In [10]:
main_symbol_series = symbol_df.index
main_symbol_series

Index(['PERI', 'TSLA', 'VOO', 'UAE', 'XLK', 'BTC-USD', 'VOOG', 'NOBL', 'SPY',
       'VHT', 'NVDA', 'SHOP', 'LUX', 'META', 'MSFT', 'IBB', 'SPHY', 'STIP',
       'USRT', 'BA', 'ADBE', 'BKNG', 'CAT', 'EL', 'DE', 'COIN', 'OCSL', 'TPVG',
       'GSBD', 'LLY', 'NVO', 'CLM', 'CEFS', 'GOF', 'SPXL', 'TQQQ', 'BRW',
       'MOAT', 'AMZN', 'INDA', 'KSA', 'SPSM', 'VBK', 'IJR', 'PAK', 'MCHI',
       'EWJ', 'MBB', 'ARKK', 'IPO', 'VXUS', 'XLF', 'ARM', 'ROM', 'RYAAY',
       'APO', 'KKR', 'JPMB', 'SMOT', 'XLV', 'BX', 'BRK-B', 'ALDAR', 'BURJEEL',
       'CHADX15', 'IHC', 'EMAAR', 'MULTIPLY', 'ASM', 'AMR'],
      dtype='object', name='Symbol')

In [11]:
computed_symbol_series = portfolio.get_return_summary().index

computed_symbol_series

Index(['VOO', 'XLK', 'VOOG', 'META', 'SPY', 'MSFT', 'NVDA', 'EMAAR', 'COIN',
       'MOAT', 'NVO', 'CHADX15', 'GOF', 'AMZN', 'TSLA', 'BRW', 'VHT', 'NOBL',
       'BTC-USD', 'IPO', 'XLF', 'CLM', 'VXUS', 'CEFS', 'RYAAY', 'CAT', 'ARKK',
       'ALDAR', 'IHC', 'LUX', 'MCHI', 'LLY', 'USRT', 'BKNG', 'VBK', 'IJR',
       'SPXL', 'SHOP', 'EWJ', 'SPSM', 'IBB', 'SPHY', 'PERI', 'JPMB', 'STIP',
       'SMOT', 'KKR', 'MBB', 'KSA', 'BA', 'INDA', 'AMR', 'ADBE', 'GSBD', 'UAE',
       'ASM', 'MULTIPLY', 'XLV', 'BRK-B', 'DE', 'OCSL', 'BX', 'ARM', 'BURJEEL',
       'TQQQ', 'EL', 'ROM', 'TPVG'],
      dtype='object')

In [12]:
[x for x in main_symbol_series if x not in computed_symbol_series]

['PAK', 'APO']

In [13]:
total_returns = (
    portfolio.holdings["unrealized_gains"]
    + portfolio.holdings["realized_gains"]
    + portfolio.holdings["income"]
)

In [14]:
fig_holdings = go.Figure()

for symbol in total_returns.columns:
    fig_holdings.add_trace(
        go.Scatter(
            x=total_returns.index,
            y=total_returns[symbol],
            mode="lines",
            name=symbol,
            line=dict(width=1),
        )
    )

fig_holdings.update_layout(
    title_text="<b>Individual Holding Performance Over Time</b>",
    template="plotly_white",
    xaxis_title="Date",
    yaxis_title=f"Total Returns ({config.BASE_CURRENCY})",
)

fig_holdings.show()

### Initialize and Run the Benchmark Simulation

In [15]:
# Create an instance of the Benchmark class
benchmark_simulation = Benchmark(master_log, date_range, last_market_day)

# Run benchmark simulation
benchmark_simulation.run_simulation()

### Display Benchmark Result

In [16]:
benchmark_results = benchmark_simulation.get_results()
benchmark_results

,Open,Close,Dividends,Market,NetDeposit,Shares,DividendCash,TradeCash,Commission,NetDividend,PortfolioValue,TotalValue,TradeTrigger
Date,,,,,,,,,,,,,
2023-02-16,375.660004,375.190002,0.0,Open,271.57,0.000000,0.0,271.57,0.0,0.0,0.000000,0.000000,Buy
2023-02-17,373.179993,374.220001,0.0,Open,0.00,0.725039,0.0,0.00,1.0,0.0,271.324047,271.324047,None
2023-02-18,373.179993,374.220001,0.0,Closed,0.00,0.725039,0.0,0.00,0.0,0.0,271.324047,271.324047,None
2023-02-19,373.179993,374.220001,0.0,Closed,0.00,0.725039,0.0,0.00,0.0,0.0,271.324047,271.324047,None
2023-02-20,373.179993,374.220001,0.0,Closed,0.00,0.725039,0.0,0.00,0.0,0.0,271.324047,271.324047,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-04,575.169983,580.140015,0.0,Open,244.41,6.564363,0.0,0.00,1.0,0.0,3808.249670,3808.249670,None
2025-08-05,580.789978,577.349976,0.0,Open,0.00,6.564363,0.0,0.00,0.0,0.0,3789.934841,3789.934841,None
2025-08-06,578.250000,581.640015,0.0,Open,0.00,6.564363,0.0,0.00,0.0,0.0,3818.096214,3818.096214,None


Ensure that if base currency is AED or INR and the benchmark prices are in non-base currency, that all benchmark prices are converted to base currency.

### Visualize Portfolio & Benchmark Performance

In [17]:
monthly_deposits = benchmark_results["NetDeposit"].resample("ME").sum()

fig = make_subplots(specs=[[{"secondary_y": True}]])


# Personal Portfolio Value
fig.add_trace(
    go.Scatter(
        x=portfolio_value.index,
        y=portfolio_value,
        mode="lines",
        name="Personal Portfolio",
        line=dict(color="green", width=2),
    ),
    secondary_y=False,
)

# Benchmark Value
fig.add_trace(
    go.Scatter(
        x=benchmark_results.index,
        y=benchmark_results["TotalValue"],
        mode="lines",
        name=f"{config.BENCHMARK_INDEX} Benchmark",
        line=dict(color="red", width=2),
    ),
    secondary_y=False,
)

# Cumulative Net Deposits
fig.add_trace(
    go.Scatter(
        x=benchmark_results.index,
        y=benchmark_results["NetDeposit"].cumsum(),
        mode="lines",
        name="Cumulative Net Deposits",
        line=dict(color="darkgrey", width=1, dash="dash"),
    ),
    secondary_y=False,
)

# Monthly Deposits/Withdrawals (on secondary y-axis)
fig.add_trace(
    go.Bar(
        x=monthly_deposits.index,
        y=monthly_deposits,
        name="Deposits / Withdrawals",
        marker_color="royalblue",
        opacity=0.4,
    ),
    secondary_y=True,
)

fig.update_layout(
    title_text=f"Portfolio Performance vs. {config.BENCHMARK_INDEX} Benchmark",
    template="plotly_white",
    barmode="relative",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    xaxis=dict(
        title="Date",
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1y", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    ),
)

fig.update_yaxes(
    title_text=f"<b>Portfolio Value ({config.BASE_CURRENCY})</b>", secondary_y=False
)
fig.update_yaxes(
    title_text="<b>Monthly Cash Flow</b>",
    secondary_y=True,
    showgrid=False,
    layer="below traces",
)

fig.show()

In [18]:
portfolio_income = portfolio.get_monthly_income()
benchmark_income = benchmark_simulation.get_monthly_income()

fig_income_comp = go.Figure()

fig_income_comp.add_trace(
    go.Bar(
        x=portfolio_income.index,
        y=portfolio_income,
        name="Portfolio Income",
        marker_color="mediumseagreen",
    )
)

fig_income_comp.add_trace(
    go.Bar(
        x=benchmark_income.index,
        y=benchmark_income,
        name=f"{config.BENCHMARK_INDEX} Benchmark Income",
        marker_color="grey",
    )
)

fig_income_comp.update_layout(
    title_text="<b>Monthly Income Comparison</b>",
    template="plotly_white",
    barmode="group",
    xaxis_title="Date",
    yaxis_title=f"Net Income ({config.BASE_CURRENCY})",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
)

fig_income_comp.show()